# Introduction
Name: Manit Gandhi \
ID:6280004


# About the project and my experience of machine learning
Machine learning-based. \
I have never tried machine learning before. \
Use powerful python libraries that I have never heard of. \
Useful given the situation of the pandemic. \
LEARN MACHINE-LEARNING. 



## Imports

In [5]:
#OpenCV is a great tool for image processing and performing computer vision tasks
import cv2
#tensorflow and keras will be used for machine learning
import tensorflow
import keras
#Pillow is a library that works with videos and images
from PIL import Image
import numpy as np

## Webcam Set-Up

In [6]:
#Use 0, 1 or 2 depending on your device. If 0 does not work change to 1
webcam = cv2.VideoCapture(1)

In [7]:
#webcam.read() returns whether the read was successful
# It also returns an array of rgb numbers for each pixel
#Both of the values are stored in success(boolean) and image_bgr(array of numbers denoting color for each pixel)
success, image_bgr = webcam.read()

In [8]:
#That is an XML file containing face detection algorithmin the OpenCV library. 
face_cascade = "/Users/manitgandhi/opt/anaconda3/lib/python3.7/site-packages/cv2/data/haarcascade_frontalface_default.xml"

In [ ]:
#What is actually happening inside the loop is that the web-cam is taking one pic in each iteration
#But since its an infinite loop, the web-cam will take multiple pics(just like a video)
while True:
    success, image_bgr = webcam.read()
    
    #Convert to black and white
    image_bw = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
    
    #use face_cascade to deetect number of faces detected and store in variable faces
    face_classifier = cv2.CascadeClassifier(face_cascade)
    faces = face_classifier.detectMultiScale(image_bw)
    
    print(f'Number of faces are {len(faces)}.')
    
    for face in faces:
        x, y, w, h = face
        cv2.rectangle(image_bgr, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
    cv2.imshow("Faces found", image_bgr)
    cv2.waitKey(1)
    

So far, our program can only detect faces. But now it is time to make it recognize if a person is wearing a mask. Therefore I have to create a classifier and use that to train the learning model.

## Creating a Classification Model
For the learning part. I will be using Teachable Machine. It is a website that allows anyone to. After that I received the code using tensor flow.  

## Using the trained model and testing it on an image(without mask)

In [14]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


np.set_printoptions(suppress=True)

# Load the model

model = tensorflow.keras.models.load_model('keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open('no_mask.jpg')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image
image.show()

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)

[[0.00000004 1.        ]]


## Testing it on an image with mask

In [ ]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = tensorflow.keras.models.load_model('keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open('yes_mask.jpg')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image
# แสดงผลรูป
image.show()

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)

## Applying the model to live feed

In [9]:
#Reading from the Webcam
webcam = cv2.VideoCapture(1)
success, image_bgr = webcam.read()

In [10]:
#Converting image from BRG to RGB and cropping it
image_rgb = cv2.cvtColor(image_bgr,cv2.COLOR_BGR2RGB)


In [11]:
cface_rgb = Image.fromarray(image_rgb)

## Mask Detection

In [12]:
face_classifier = cv2.CascadeClassifier(face_cascade)
np.set_printoptions(suppress=True)
model = tensorflow.keras.models.load_model('keras_model.h5')
size = (224, 224)

In [ ]:
while True:
    success, image_bgr = webcam.read()
    image_org = image_bgr.copy()
    image_bw = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
    image_rgb = cv2.cvtColor(image_bgr,cv2.COLOR_BGR2RGB)
    faces = face_classifier.detectMultiScale(image_bw)
    for face in faces:
        x, y, w, h = face
        cface_rgb = Image.fromarray(image_rgb[y:y+h,x:x+w])
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        image = cface_rgb
        image = ImageOps.fit(image, size, Image.ANTIALIAS)
        image_array = np.asarray(image)
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
        data[0] = normalized_image_array
        prediction = model.predict(data)
        print(prediction)
        if prediction[0][0] > prediction[0][1]:
            cv2.putText(image_bgr,'Mask Detected',(x,y-7),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
            cv2.rectangle(image_bgr, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            cv2.putText(image_bgr,'No Mask',(x,y-7),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
            cv2.rectangle(image_bgr, (x, y), (x+w, y+h), (0,0,255), 2)
 
    cv2.imshow("Mask Detection", image_bgr)
    cv2.waitKey(1) 


[[0.00000001 1.        ]]
[[0.00000001 1.        ]]
[[0.00000002 1.        ]]
[[0.00000002 1.        ]]
[[0.00000001 1.        ]]
[[0.00000001 1.        ]]
[[0.00000032 0.99999964]]
[[0.00000011 0.9999999 ]]
[[0.00000016 0.9999999 ]]
[[0.00000038 0.99999964]]
[[0.00000001 1.        ]]
[[0.00000001 1.        ]]
[[0.00000002 1.        ]]
[[0. 1.]]
[[0. 1.]]
[[0.00000001 1.        ]]
[[0.00000003 1.        ]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0.00027307 0.99972695]]
[[0. 1.]]
[[0. 1.]]
[[0.00000002 1.        ]]
[[0.00000004 1.        ]]
[[0.00000003 1.        ]]
[[0.00004731 0.9999527 ]]
[[0.00000006 1.        ]]
[[0.00000005 1.        ]]
[[0. 1.]]
[[0.00000046 0.9999995 ]]
[[0.00000923 0.9999908 ]]
[[0.00008853 0.9999114 ]]
[[0.3278191 0.6721809]]
[[0.00153747 0.99846256]]
[[0.9999937  0.00000627]]
[[0.867541   0.13245896]]
[[0.5307541 0.4692459]]
[[0.28399843 0.7160015 ]]
[[0.00001103 0.99998903]]
[[0.00008348 0.99991655]]
[[0.00005769 0.9999423 ]]
[[0.00000147 0.99999857]]
[[0.